# Daily Gas Prices in US

### Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
import math

## Data Pull & Cleaning

### API Key

In [2]:
#API_Keys
gas_key = 
g_key = 
eia_key = 

### Daily Gas Prices - Pull & Clean UP

In [3]:
#import Gas Daily Prices
url = "https://api.collectapi.com/gasPrice/allUsaPrice"

headers = {
    'content-type': "application/json",
    'authorization': f"apikey {gas_key}"
    }

response = requests.request("GET", url, headers=headers).json()

#Defining Variables
states = []
gasoline=[]
midGrade=[]
premium=[]
diesel=[]

#Creating for cycle to extract data

for x in response["result"]:
    states.append(x["name"])
    gasoline.append(x["gasoline"])
    midGrade.append(x["midGrade"])
    premium.append(x["premium"])
    diesel.append(x["diesel"])
    
#create dataframe
daily_gasoline_df = pd.DataFrame({"State":states,"Gas":gasoline,"MidGrade":midGrade,"Premium":premium,"Diesel":diesel})


In [4]:
#Change to FLOAT
daily_gasoline_df['Gas'] = daily_gasoline_df['Gas'].astype(float)
daily_gasoline_df['MidGrade'] = daily_gasoline_df['MidGrade'].astype(float)
daily_gasoline_df['Premium'] = daily_gasoline_df['Premium'].astype(float)
daily_gasoline_df['Diesel'] = daily_gasoline_df['Diesel'].astype(float)
daily_gasoline_df.head()

,State,Gas,MidGrade,Premium,Diesel
0,Alaska,2.420,2.516,2.672,2.389
1,Alabama,1.790,2.124,2.446,2.279
2,Arkansas,1.794,2.070,2.354,2.127
3,Arizona,2.286,2.553,2.808,2.505
4,California,2.989,3.161,3.276,3.284


### Google GeoCoding State

In [5]:
#import State Information
state_lat = []
state_lng = []
state_code = []


for state in states:
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(state, g_key)
    geo_data = requests.get(target_url).json()
    state_lat.append(geo_data['results'][0]["geometry"]["location"]["lat"])
    state_lng.append(geo_data['results'][0]["geometry"]["location"]["lng"])
    code = geo_data['results'][0]["address_components"][0]["short_name"]
    
    #correct to the State when compared with the City Name
    if len(code)>2:
        state_code.append(geo_data['results'][0]["address_components"][2]["short_name"])
    else:
        state_code.append(code)

#Change US to NY

state_code[34]= "NY"

#create dataframe
state_df = pd.DataFrame({"State":states,"State Code":state_code,
                         "State Latitude": state_lat, 
                         "State Longitude": state_lng})
state_df.head()

,State,State Code,State Latitude,State Longitude
0,Alaska,AK,64.200841,-149.493673
1,Alabama,AL,32.318231,-86.902298
2,Arkansas,AR,35.201050,-91.831833
3,Arizona,AZ,34.048928,-111.093731
4,California,CA,36.778261,-119.417932


### Gas Price Time Series

In [6]:
#import Gas Time Series
url = "http://api.eia.gov/series/?api_key="+eia_key+"&series_id=PET.EMM_EPM0_PTE_NUS_DPG.W"

response = requests.get(url).json()

timeSeries = response["series"][0]["data"]

#### Reduction to Wks 52

In [7]:
last52wks = timeSeries[0:52]
gasprice = []
wk = []
n=0
for week in last52wks:
    gasprice.append(week[1])
    wk.append(n)
    n=n-1

weekprices = pd.DataFrame({"WeekNum":wk,
                          "Price":gasprice})
weekprices.tail()

,WeekNum,Price
47,-47,2.860
48,-48,2.827
49,-49,2.798
50,-50,2.741
51,-51,2.759


### GDP per State

In [8]:
#import Info
gdp_df = pd.read_csv("Resources/GDP_sheet1.csv")

#Drop Rank
gdp_df = gdp_df.drop(columns="rank")
gdp_df = gdp_df.rename(columns={"US States": "State",
                               "GDP per capita (current dollars)":"GDP per capita"})
gdp_df['GDP per capita']=gdp_df['GDP per capita'].str.replace(",","").astype(float)
gdp_df.head()

,State,GDP per capita
0,District of Columbia,200277.0
1,New York,85746.0
2,Massachusetts,82480.0
3,Delaware,77253.0
4,Connecticut,76342.0


### US Refinaries

In [9]:
#import Info
refineries_df = pd.read_csv("Resources/Refineries.csv")
refineries_df = refineries_df.rename(columns={"Unnamed: 1":"State"})
refineries_df = refineries_df.rename(columns={"List of oil refineries in the US":"Refinery"})
refineries_df["Refinery"] = refineries_df["Refinery"].str.replace(r"\(.*\)","")
refineries_df = refineries_df.drop_duplicates()
refineries_df = refineries_df.drop_duplicates(subset="Refinery",keep=False)
refineries_perState = refineries_df.groupby("State").count()
refineries_df = refineries_df.merge(state_df,left_on="State",right_on="State")
refineries_grouped = refineries_df.groupby("State Code").count()
refineries_perState = refineries_grouped.reset_index()
refineries_perState = refineries_perState.drop(columns=["State","State Latitude","State Longitude"])
refineries_perState.replace("",np.nan)
refineries_perState.head()


,State Code,Refinery
0,AK,4
1,AL,3
2,AR,1
3,CA,16
4,CO,1


In [10]:
# geocoordinates
target_type = "refinery"

refinery_name = []
refinery_lat=[]
refinery_lng = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

for i in range(0,len(refineries_df["State"])):
    
    target_search = refineries_df.iloc[i][0]
    # set up a parameters dictionary
    params = {
        "query": target_search,
        "key": g_key}
    refinery_name.append(target_search)
    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=params).json()
        refinery_lat.append(response["results"][0]["geometry"]["location"]["lat"])
        refinery_lng.append(response["results"][0]["geometry"]["location"]["lng"])
    except:
        refinery_lat.append("")
        refinery_lng.append("")

In [11]:
refinery_place_df = pd.DataFrame({"Refinery":refinery_name,
                                 "Refinery Lat":refinery_lat,
                                 "Refinery Lng": refinery_lng})

In [12]:
refinery_place_df = refinery_place_df.replace("",np.nan)
refinery_place_df = refinery_place_df.dropna()
refinery_place_df.head()

,Refinery,Refinery Lat,Refinery Lng
0,Atmore Refinery,31.181033,-87.439461
1,Saraland Refinery,30.807599,-88.064032
2,Tuscaloosa Refinery,33.197127,-87.604946
3,Kenai Refinery,60.686568,-151.369513
4,North Pole Refinery,64.733348,-147.343500


### Calculating distance between Refinaries and States

In [13]:
total = dict()
ref_name = refinery_place_df["Refinery"].to_list()
ref_lat = refinery_place_df["Refinery Lat"].to_numpy()
ref_lng = refinery_place_df["Refinery Lng"].to_numpy()
stt_lat = state_df["State Latitude"].to_numpy()
stt_lng = state_df["State Longitude"].to_numpy()
stt_code = state_df["State Code"].to_list()

for j in range(0,len(refinery_place_df["Refinery"])):
    distances=[]
    point2lat = ref_lat[j]
    point2lng = ref_lng[j]
    for i in range(0,len(state_df["State Code"])):
        point1lat = stt_lat[i]
        point1lng = stt_lng[i]
        r= 6371000 #earth Radius
        phi1 = point1lat*(math.pi/180)
        phi2 = point2lat*(math.pi/180)
        deltaphi=(point2lat - point1lat)*(math.pi/180)
        deltaalpha = (point2lng - point1lng)*(math.pi/180)
        a = math.sin(deltaphi/2)*math.sin(deltaphi/2) + math.cos(phi1)*math.cos(phi2)*math.sin(deltaalpha)*math.sin(deltaalpha)
        c = 2* math.atan2(math.sqrt(a),math.sqrt(1-a))
        d = r*c
        distances.append(d)
    total.update({ref_name[j]:distances})

total.update({"State Code":stt_code})
distance_df=pd.DataFrame.from_dict(total)
distance_df.columns

Index(['Atmore Refinery ', 'Saraland Refinery ', 'Tuscaloosa Refinery ',
       'Kenai Refinery ', 'North Pole Refinery ', 'Prudhoe Bay Refinery ',
       'Valdez Refinery ', 'Smackover Refinery ', 'Bakersfield Refinery ',
       'Benicia Refinery ',
       ...
       'Ferndale Refinery ', 'Tacoma Refinery ', 'Newell Refinery ',
       'Superior Refinery ', 'Cheyenne Refinery ', 'Evanston Refinery ',
       'Newcastle Refinery ', 'Sinclair Wyoming Refinery ',
       'Sinclair Casper Refinery ', 'State Code'],
      dtype='object', length=120)

In [14]:
distance_df.set_index(["State Code"], inplace = True, 
                    append = True, drop = True)
distance_df.head()

,,Atmore Refinery,Saraland Refinery,Tuscaloosa Refinery,Kenai Refinery,North Pole Refinery,Prudhoe Bay Refinery,Valdez Refinery,Smackover Refinery,Bakersfield Refinery,Benicia Refinery,...,Cherry Point Refinery,Ferndale Refinery,Tacoma Refinery,Newell Refinery,Superior Refinery,Cheyenne Refinery,Evanston Refinery,Newcastle Refinery,Sinclair Wyoming Refinery,Sinclair Casper Refinery
,State Code,,,,,,,,,,,,,,,,,,,,,
0,AK,8.349362e+06,8.343113e+06,8.129890e+06,4.356547e+05,2.143971e+05,6.919049e+05,4.793793e+05,7.730457e+06,5.120246e+06,4.573604e+06,...,3.553349e+06,3.561757e+06,3.723685e+06,7.786714e+06,6.451242e+06,5.942992e+06,7.220355e+06,5.741625e+06,5.669740e+06,5.650218e+06
1,AL,1.621978e+05,2.768906e+05,1.637642e+05,8.684645e+06,8.077501e+06,7.765723e+06,8.253966e+06,1.091530e+06,5.839700e+06,6.282592e+06,...,6.083547e+06,6.079474e+06,6.069492e+06,1.449786e+06,1.823767e+06,3.309478e+06,1.085356e+06,3.260408e+06,3.700255e+06,3.568125e+06
2,AR,9.311395e+05,8.553921e+05,8.084738e+05,8.011348e+06,7.412331e+06,7.172682e+06,7.544580e+06,2.607606e+05,4.874098e+06,5.321230e+06,...,5.171447e+06,5.166909e+06,5.145954e+06,2.049287e+06,1.278111e+06,2.349784e+06,1.037165e+06,2.319811e+06,2.741949e+06,2.616055e+06
3,AZ,4.403158e+06,4.301771e+06,4.311624e+06,6.260617e+06,5.806169e+06,5.940845e+06,5.705529e+06,3.382681e+06,1.459899e+06,2.031270e+06,...,2.527799e+06,2.519811e+06,2.391776e+06,5.329320e+06,3.474175e+06,1.359096e+06,4.134308e+06,1.609567e+06,1.107152e+06,1.291443e+06
4,CA,5.821029e+06,5.730100e+06,5.701791e+06,5.115889e+06,4.762700e+06,5.050154e+06,4.570002e+06,4.812218e+06,1.675813e+05,5.025810e+05,...,1.449713e+06,1.441607e+06,1.263778e+06,6.519717e+06,4.574127e+06,2.562949e+06,5.352567e+06,2.678195e+06,2.182419e+06,2.338642e+06


In [15]:
mean_distance = distance_df.mean(axis=1)/1000
min_distance = distance_df.min(axis=1)/1000
state_refinery_distance = pd.DataFrame({"Mean Distance (km)":mean_distance,
                                        "Min Distance (km)":min_distance})

state_refinery_distance =state_refinery_distance.reset_index()
state_refinery_distance = state_refinery_distance.drop(columns = "level_0")
state_refinery_distance.head()

,State Code,Mean Distance (km),Min Distance (km)
0,AK,6679.875943,214.397078
1,AL,2878.048861,162.197776
2,AR,2449.591909,260.760607
3,AZ,3335.455721,512.568073
4,CA,4095.681542,167.581304


### Auto per State Information

In [16]:
autos_df = pd.read_csv("Resources/autosbystate.csv")
autos_df = autos_df.rename(columns={"STATE": "State"})

In [17]:
#Calculate Total # of Motor vehicles

autos_df['TOTAL MOTOR VEHICLES'] = autos_df['TOTAL AUTOMOBILES'] + autos_df['TOTAL BUSES'] + autos_df['TOTAL TRUCKS']+ autos_df['TOTAL MOTORCYCLES']

In [18]:
autos_df.head()

,State,PRIVATE AND COMMERCIAL AUTOMOBILE,PUBLIC AUTOMOBILE,TOTAL AUTOMOBILES,PRIVATE AND COMMERCIAL BUSES,PUBLIC BUSES,TOTAL BUSES,PRIVATE AND COMMERCIAL TRUCKS,PUBLIC TRUCKS,TOTAL TRUCKS,PRIVATE AND COMMERCIAL MOTORCYCLES,PUBLIC MOTORCYCLES,TOTAL MOTORCYCLES,TOTAL MOTOR VEHICLES
0,Alabama,2021395,44068,2065463,5550,132,5682,2806624,66119,2872743,111247,938,112185,5056073
1,Alaska,169412,4075,173487,7496,584,8080,573399,17680,591079,31852,7,31859,804505
2,Arizona,2371199,20425,2391624,1508,7543,9051,3356363,43341,3399704,164053,2,164055,5964434
3,Arkansas,914265,12524,926789,11229,712,11941,1778174,27336,1805510,89457,—,89457,2833697
4,California,14615499,245468,14860967,35253,64664,99917,14693888,297826,14991714,828083,14460,842543,30795141


### Merging Information

In [19]:
daily_gasoline_df = daily_gasoline_df.merge(state_df,left_on="State",right_on="State")
daily_gasoline_df = daily_gasoline_df.merge(gdp_df,on="State",how="left")
daily_gasoline_df = daily_gasoline_df.merge(state_refinery_distance,on="State Code",how="left")
daily_gasoline_df = daily_gasoline_df.merge(refineries_perState,on="State Code",how="left")
daily_gasoline_df = daily_gasoline_df.rename(columns={"Refinery": "Refinery Count"})
daily_gasoline_df = daily_gasoline_df.merge(autos_df,on="State",how="left")
daily_gasoline_df.columns

Index(['State', 'Gas', 'MidGrade', 'Premium', 'Diesel', 'State Code',
       'State Latitude', 'State Longitude', 'GDP per capita',
       'Mean Distance (km)', 'Min Distance (km)', 'Refinery Count',
       'PRIVATE AND COMMERCIAL AUTOMOBILE', 'PUBLIC AUTOMOBILE',
       'TOTAL AUTOMOBILES', 'PRIVATE AND COMMERCIAL BUSES', 'PUBLIC BUSES',
       'TOTAL BUSES', 'PRIVATE AND COMMERCIAL TRUCKS', 'PUBLIC TRUCKS',
       'TOTAL TRUCKS', 'PRIVATE AND COMMERCIAL MOTORCYCLES',
       'PUBLIC MOTORCYCLES', 'TOTAL MOTORCYCLES', 'TOTAL MOTOR VEHICLES'],
      dtype='object')

In [20]:
daily_gasoline_df.head()

,State,Gas,MidGrade,Premium,Diesel,State Code,State Latitude,State Longitude,GDP per capita,Mean Distance (km),...,PRIVATE AND COMMERCIAL BUSES,PUBLIC BUSES,TOTAL BUSES,PRIVATE AND COMMERCIAL TRUCKS,PUBLIC TRUCKS,TOTAL TRUCKS,PRIVATE AND COMMERCIAL MOTORCYCLES,PUBLIC MOTORCYCLES,TOTAL MOTORCYCLES,TOTAL MOTOR VEHICLES
0,Alaska,2.420,2.516,2.672,2.389,AK,64.200841,-149.493673,73205.0,6679.875943,...,7496,584,8080,573399,17680,591079,31852,7,31859,804505
1,Alabama,1.790,2.124,2.446,2.279,AL,32.318231,-86.902298,45219.0,2878.048861,...,5550,132,5682,2806624,66119,2872743,111247,938,112185,5056073
2,Arkansas,1.794,2.070,2.354,2.127,AR,35.201050,-91.831833,42454.0,2449.591909,...,11229,712,11941,1778174,27336,1805510,89457,—,89457,2833697
3,Arizona,2.286,2.553,2.808,2.505,AZ,34.048928,-111.093731,48055.0,3335.455721,...,1508,7543,9051,3356363,43341,3399704,164053,2,164055,5964434
4,California,2.989,3.161,3.276,3.284,CA,36.778261,-119.417932,74205.0,4095.681542,...,35253,64664,99917,14693888,297826,14991714,828083,14460,842543,30795141


### Exporting all information

In [21]:
daily_gasoline_df.to_csv("Data/working_data.csv",index=False)
refinery_place_df.to_csv("Data/refinery.csv",index=False)
weekprices.to_csv("Data/historicalGasPrice.csv",index=False)